In [1]:
import json
import random
import pickle
import numpy as np

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer

In [3]:
import tensorflow as tf
from tensorflow.keras import layers

In [4]:
Lemmatizer = WordNetLemmatizer()

In [5]:
intents = json.loads(open('intent.json').read())

In [17]:
words = []
classes = []
documents = []
ignore = ['!', '@', '#', '$', '%', '&', '(', ')',
          '-,' '_', '[,' '],' '{', '}',
          ';', ':', '.', '/', '<', '>', '?', "'"]

In [294]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))    
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [295]:
responses = {}
for intent in intents['intents']:
    responses[intent['tag']] = intent['responses']

In [296]:
responses['greetings'][0]

'Hello'

In [297]:
words = [Lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore]

In [301]:
len(classes)

12

In [302]:
words = sorted(set(words))  #save words
classes = sorted(set(classes))

In [303]:
def imp_bag(string): #word to string helps me to covert
    string = nltk.word_tokenize(string)
    bag = []
    word_pattern = [Lemmatizer.lemmatize(word.lower()) for word in words]
    for word in words:
        bag.append(1) if word in string else  bag.append(0)
    return bag
    

In [304]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [305]:
training = []
input_em = [0]*len(words)
output_em = [0]*len(classes)

In [322]:
classes

['bestcrop',
 'cropprice',
 'dieseases',
 'farmingstuff',
 'fertiliser ',
 'goodbye',
 'greetings',
 'loans',
 'name',
 'noanswer',
 'soil',
 'thanks']

In [307]:
for index, document in enumerate(documents):
    bag = []
    word_pattern = document[0]
    word_pattern = [Lemmatizer.lemmatize(word.lower()) for word in word_pattern]
    for word in words:
        bag.append(1) if word in word_pattern else  bag.append(0)
    output_row = list(output_em)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [309]:
training = np.array(training)

C:\Users\garim\AppData\Local\Temp\ipykernel_5252\3578359385.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [310]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [311]:
model = tf.keras.Sequential()
model.add(layers.Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(len(train_y[0]), activation='softmax'))


In [312]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x,
                                                   train_y,
                                                   test_size=0.2, random_state=42)

In [313]:
optimizer = tf.keras.optimizers.SGD()

In [314]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [315]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1, validation_data=(X_test, y_test))

Epoch 1/200
42/42 [==============================] - 1s 17ms/step - loss: 2.4720 - accuracy: 0.1095 - val_loss: 2.4299 - val_accuracy: 0.2381
Epoch 2/200
42/42 [==============================] - 0s 10ms/step - loss: 2.4197 - accuracy: 0.2286 - val_loss: 2.3890 - val_accuracy: 0.3571
Epoch 3/200
42/42 [==============================] - 0s 10ms/step - loss: 2.3761 - accuracy: 0.2667 - val_loss: 2.3469 - val_accuracy: 0.2857
Epoch 4/200
42/42 [==============================] - 0s 10ms/step - loss: 2.3328 - accuracy: 0.3095 - val_loss: 2.3022 - val_accuracy: 0.3571
Epoch 5/200
42/42 [==============================] - 0s 10ms/step - loss: 2.2671 - accuracy: 0.3952 - val_loss: 2.2549 - val_accuracy: 0.3571
Epoch 6/200
42/42 [==============================] - 0s 10ms/step - loss: 2.2282 - accuracy: 0.3857 - val_loss: 2.2061 - val_accuracy: 0.4286
Epoch 7/200
42/42 [==============================] - 0s 10ms/step - loss: 2.1647 - accuracy: 0.3952 - val_loss: 2.1519 - val_accuracy: 0.4286
Epoch 

42/42 [==============================] - 0s 10ms/step - loss: 0.2013 - accuracy: 0.9857 - val_loss: 0.0949 - val_accuracy: 1.0000
Epoch 59/200
42/42 [==============================] - 0s 10ms/step - loss: 0.2056 - accuracy: 0.9857 - val_loss: 0.0888 - val_accuracy: 1.0000
Epoch 60/200
42/42 [==============================] - 0s 10ms/step - loss: 0.2029 - accuracy: 0.9905 - val_loss: 0.0840 - val_accuracy: 1.0000
Epoch 61/200
42/42 [==============================] - 0s 10ms/step - loss: 0.2131 - accuracy: 0.9524 - val_loss: 0.0791 - val_accuracy: 1.0000
Epoch 62/200
42/42 [==============================] - 0s 10ms/step - loss: 0.1777 - accuracy: 0.9905 - val_loss: 0.0753 - val_accuracy: 1.0000
Epoch 63/200
42/42 [==============================] - 0s 10ms/step - loss: 0.1598 - accuracy: 0.9857 - val_loss: 0.0712 - val_accuracy: 1.0000
Epoch 64/200
42/42 [==============================] - 0s 11ms/step - loss: 0.1644 - accuracy: 0.9810 - val_loss: 0.0672 - val_accuracy: 1.0000
Epoch 65/200

KeyboardInterrupt: 

In [ ]:
classes

In [318]:
A = imp_bag("profit")
(model.predict((A,)))

1/1 [==============================] - 0s 108ms/step


array([[0.00649686, 0.11789992, 0.01264151, 0.00294254, 0.00593849,
        0.07105346, 0.6035748 , 0.03626235, 0.03446772, 0.00303136,
        0.0288733 , 0.07681761]], dtype=float32)

In [328]:
i = 0
while(True):
    if(i == 0):
        print("Hello, I'm Tim, how may I help you")
    st = input()
    st = imp_bag(st)
    cl = np.argmax(model.predict((st,)))
    if(np.amax(model.predict((st,))) < 0.7):
        print("ma chuda bosdu")
        i = i+1
        continue
    print(np.amax(model.predict((st,))))
    ans = classes[cl]
    print(responses[ans][0])
    i = i+1
    print("-"*100)
    

Hello, I'm Tim
hjsdfjhn sjh ndfjsndfjh sd
1/1 [==============================] - 0s 52ms/step
ma chuda bosdu
hello
1/1 [==============================] - 0s 51ms/step
0.9794845
Hello
----------------------------------------------------------------------------------------------------


KeyboardInterrupt: Interrupted by user

In [ ]:
adios